## Week 3 Assignment: Segmenting, part three
## Explore and cluster the neighborhoods in Toronto

In [1]:
#Loading Libraries
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
#Fetching Postal code of Canada and transform the data into a pandas dataframe 
canada_postal_df  = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
canada_postal_df .rename(columns = {'Postcode':'PostalCode'},inplace=True)

In [3]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
canada_postal_df.drop(canada_postal_df[canada_postal_df['Borough']=="Not assigned"].index,axis=0, inplace=True)

In [4]:
#Gouping More than one neighborhood that has the same Borough name
canada_postal_df=canada_postal_df.groupby("PostalCode", as_index=False).agg(lambda x:','.join(set(x)))

In [5]:
#Assign Not assigned neighborhood name by Borough name
canada_postal_df.loc[canada_postal_df['Neighbourhood']=="Not assigned",'Neighbourhood']=canada_postal_df.loc[canada_postal_df['Neighbourhood']=="Not assigned",'Borough']

In [6]:
geo_data=pd.read_csv("https://cocl.us/Geospatial_data")

In [7]:
canada_postal_df['Latitude']=geo_data['Latitude'].values
canada_postal_df['Longitude']=geo_data['Longitude'].values

In [8]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

%matplotlib inline  

print('Libraries imported.')

Libraries imported.


In [9]:
# printing number of borough and the neighborhoods
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(canada_postal_df['Borough'].unique()),
        canada_postal_df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [10]:
# Cleaning the dataframe and keep only Borough, Neighbourhood, latitude and longitude columns
neighborhoods = canada_postal_df.drop(columns=['PostalCode'], axis=1)

In [11]:
# work with only boroughs that contain the word Toronto 
neighborhoods = neighborhoods[neighborhoods['Borough'].str.contains("Toronto")]

In [12]:
# setting the address, latitude,longitude and the geocode of Toronto 
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City is 43.653963, -79.387207.


In [13]:
# create map of  using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto